In [1]:
from keras import Model
from keras import layers
from matplotlib import pyplot
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

def resnet_block(x, filters, stride = (1,1), match = False):

  shortcut = x
  x = layers.Conv2D(filters, (1,1), strides = stride)(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.Conv2D(filters, (3,3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.Conv2D(filters*4, (1,1))(x)
  x = layers.BatchNormalization()(x)

  if (match == True):#To match size of shortcut to the output if needed
    shortcut = layers.Conv2D(filters*4, (1,1), strides = stride)(shortcut)
  
  x = layers.add([x,shortcut])
  x = layers.ReLU()(x)

  return x

In [2]:
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar100.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [3]:
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [4]:
trainX, trainY, testX, testY = load_dataset()
trainX, testX = prep_pixels(trainX, testX)


169009152/169001437 [==============================] - 4s 0us/step


In [5]:
inputs = layers.Input(shape = (32,32,3))
x = layers.Conv2D(64, kernel_size=(3,3), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)


x = resnet_block(x, 64, match = True)

for _ in range(2):
  x = resnet_block(x,64)

#double the number of filters
x = resnet_block(x, 128, stride = (2,2), match = True)

for _ in range(3):
  x = resnet_block(x,128)

#double the number of filters
x = resnet_block(x, 256, stride = (2,2), match = True)

for _ in range(5):
  x = resnet_block(x,256)

x = resnet_block(x, 512, stride = (2,2), match = True)

for _ in range ( 2 ):
  x = resnet_block(x, 512)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(100, activation = 'softmax')(x)

model = Model(inputs,outputs )
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [6]:
from keras.callbacks import ModelCheckpoint
filepath='/content/sample_data/best.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history = model.fit(trainX, trainY, epochs=50, batch_size=64, validation_data=(testX, testY), callbacks=callbacks_list)
model.save('cifar model')

Epoch 1/50
782/782 [==============================] - 93s 119ms/step - loss: 4.0783 - accuracy: 0.0903 - val_loss: 11.5206 - val_accuracy: 0.1121
Epoch 2/50
782/782 [==============================] - 93s 119ms/step - loss: 3.2942 - accuracy: 0.2004 - val_loss: 5.1617 - val_accuracy: 0.2135
Epoch 3/50
782/782 [==============================] - 93s 119ms/step - loss: 2.8342 - accuracy: 0.2907 - val_loss: 8.3671 - val_accuracy: 0.1421
Epoch 4/50
782/782 [==============================] - 93s 119ms/step - loss: 2.4608 - accuracy: 0.3606 - val_loss: 2.7795 - val_accuracy: 0.3086
Epoch 5/50
782/782 [==============================] - 93s 119ms/step - loss: 2.1267 - accuracy: 0.4319 - val_loss: 2.8380 - val_accuracy: 0.2983
Epoch 6/50
782/782 [==============================] - 93s 119ms/step - loss: 1.8345 - accuracy: 0.4952 - val_loss: 2.8628 - val_accuracy: 0.3523
Epoch 7/50
782/782 [==============================] - 93s 119ms/step - loss: 1.5658 - accuracy: 0.5572 - val_loss: 2.0188 - val_a